In [2]:
import more_itertools
from typing import List
from embedding_generator_default import EmbeddingGeneratorDefault
from mongo_db_client import MongoDbClient
from pre_processer_default import PreProcesserDefault
import tensorflow as tf
import numpy as np
from models import PairDbDoc

/Users/beto/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-05-28 23:20:30.892781: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-05-28 23:20:30.892873: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
db_client = MongoDbClient()
embedding_generator = EmbeddingGeneratorDefault()
pre_processer = PreProcesserDefault()

In [4]:
def get_embeddings_from_batched_pairs(pairs: List[PairDbDoc]):
  code_sentences = [pre_processer.process_code(pair['code_tokens']) for pair in pairs]
  comment_sentences = [pre_processer.process_text(pair['comment_tokens']) for pair in pairs]

  return embedding_generator.from_code(code_sentences), embedding_generator.from_text(comment_sentences)

After tests using 64, 128 and 256 for batch_size and 1024 pairs from db, the fastest for generate embeddings was 64

In [7]:
pairs = list(db_client.get_pairs_collection().find().limit(64))
code_emb, comment_emb = get_embeddings_from_batched_pairs(pairs)


In [4]:
from keras import Sequential
from keras.layers import Input, Dense, Concatenate, Reshape
from utils import encoder_hidden_size, encoder_seq_len

transformer_embedding_shape = (encoder_seq_len, encoder_hidden_size)
hidden_activation = 'relu'
output_activation = 'sigmoid'

code_input, comment_input = Input(shape=transformer_embedding_shape, name="code_input"), Input(shape=transformer_embedding_shape, name="comment_input")
code_input, comment_input
# code_input = Reshape(target_shape=(-1, ))()
# comment_input = Reshape(target_shape=(-1, ))() 
# merged = Concatenate(name="joint_embedding")([code_input, comment_input])

# merged

(<KerasTensor: shape=(None, 512, 384) dtype=float32 (created by layer 'code_input')>,
 <KerasTensor: shape=(None, 512, 384) dtype=float32 (created by layer 'comment_input')>)

In [5]:
reshaped_code = Reshape(target_shape=(-1, ))(code_input)
reshaped_comment = Reshape(target_shape=(-1, ))(comment_input)
reshaped_code, reshaped_comment

(<KerasTensor: shape=(None, 196608) dtype=float32 (created by layer 'reshape_2')>,
 <KerasTensor: shape=(None, 196608) dtype=float32 (created by layer 'reshape_3')>)

In [6]:
merged = Concatenate(name="joint_embedding")([reshaped_code, reshaped_comment])
merged

<KerasTensor: shape=(None, 393216) dtype=float32 (created by layer 'joint_embedding')>